In [2]:
# coding: utf-8
import os, getpass

if getpass.getuser()=='jovyan':
    os.environ['KERAS_BACKEND'] = 'tensorflow'
    os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
#http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.h
#http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
#http://scikit-learn.org/stable/tutorial/basic/tutorial.html#model-persistence
from sklearn.externals import joblib
#https://en.wikipedia.org/wiki/Activation_function
from keras.models import Sequential, Model, model_from_json
from keras.optimizers import SGD
from keras.layers import Input, Activation, Dense, Convolution2D, MaxPooling2D, Dropout, Flatten
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.layers import Merge, merge
from keras import backend as K
#https://docs.scipy.org/doc/numpy/reference/generated/numpy.clip.html
import numpy as np
import pandas as pd
import sys, glob, argparse, h5py
from itertools import cycle
from scipy import interp
#from ipywidgets import FloatProgress
#from IPython.display import display
#https://github.com/tqdm/tqdm#latest-conda-release
from tqdm import trange, tqdm
from MLJEC_MCTruth_Util import rotate_and_reflect, prepare_df_dict, JetImageGenerator
import MLJEC_MCTruth_Plot as plotter

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using TensorFlow backend.


In [3]:
# get input numpy arrays
inputs = {}
inputs['QCD80'] = glob.glob('QCD_pt_80_120.npy')
inputs['QCD120'] = glob.glob('QCD_pt_120_170.npy')
inputs['QCD170'] = glob.glob('QCD_pt_170_300.npy')
list_params = {}
params = {}
for key, input_files in inputs.iteritems():
    list_params[key] = []
    for in_file in input_files:
        try:
            arr = np.load(in_file)
            list_params[key].append(arr)
        except ValueError:
            print 'bad file: %s'%in_file
    params[key] = np.concatenate(list_params[key])

In [4]:
print params['QCD80'].dtype.names

('run', 'lumi', 'event', 'met', 'sumet', 'rho', 'pthat', 'mcweight', 'njet', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_area', 'jet_jes', 'chf', 'nhf', 'phf', 'elf', 'muf', 'hf_hf', 'hf_phf', 'hf_hm', 'hf_phm', 'chm', 'nhm', 'phm', 'elm', 'mum', 'beta', 'bstar', 'ak5pfcand_pt', 'ak5pfcand_eta', 'ak5pfcand_phi', 'ak5pfcand_id', 'ak5pfcand_charge', 'ak5pfcand_ijet')


In [5]:
df_dict_jet = {}
df_dict_cand = {}

In [8]:
for QCDBin in ['QCD80','QCD120','QCD170']:
    df_dict_jet[QCDBin] = pd.DataFrame(params[QCDBin],columns=['run', 'lumi', 'event', 'met', 'sumet', 'rho', 'pthat', 'mcweight', 'njet', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_area', 'jet_jes', 'chf', 'nhf', 'phf', 'elf', 'muf', 'hf_hf', 'hf_phf', 'hf_hm', 'hf_phm', 'chm', 'nhm', 'phm', 'elm', 'mum', 'beta', 'bstar','ak5pfcand_ijet'])
    df_dict_jet[QCDBin] = df_dict_jet[QCDBin].drop_duplicates()
    df_dict_jet[QCDBin] = df_dict_jet[QCDBin][(df_dict_jet[QCDBin].jet_pt > 80)]
    df_dict_cand[QCDBin] = pd.DataFrame(params[QCDBin], columns=['event', 'jet_pt', 'jet_eta', 'jet_phi','ak5pfcand_pt', 'ak5pfcand_eta', 'ak5pfcand_phi', 'ak5pfcand_id', 'ak5pfcand_charge', 'ak5pfcand_ijet'])
    df_dict_cand[QCDBin] = df_dict_cand[QCDBin][(df_dict_cand[QCDBin].jet_pt > 80)]

In [9]:
df_dict_jets = pd.concat([df_dict_jet['QCD80'],df_dict_jet['QCD120'],df_dict_jet['QCD170']])
df_dict_cands = pd.concat([df_dict_cand['QCD80'],df_dict_cand['QCD120'],df_dict_cand['QCD170']])


In [10]:
####################
# Global Variables #
####################
nx = 30 # size of image in eta
ny = 30 # size of image in phi
xbins = np.linspace(-1.4,1.4,nx+1)
ybins = np.linspace(-1.4,1.4,ny+1)
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# rotation + (possible) reflection needed later
def rotate_and_reflect(x,y,w):
    rot_x = []
    rot_y = []
    theta = 0
    maxPt = -1
    for ix, iy, iw in zip(x, y, w):
        dv = np.matrix([[ix],[iy]])-np.matrix([[x.iloc[0]],[y.iloc[0]]])
        dR = np.linalg.norm(dv)
        thisPt = iw
        if dR > 0.35 and thisPt > maxPt:
            maxPt = thisPt
            # rotation in eta-phi plane c.f  https://arxiv.org/abs/1407.5675 and https://arxiv.org/abs/1511.05190:
            # theta = -np.arctan2(iy,ix)-np.radians(90)
            # rotation by lorentz transformation c.f. https://arxiv.org/abs/1704.02124:
            px = iw * np.cos(iy)
            py = iw * np.sin(iy)
            pz = iw * np.sinh(ix)
            theta = np.arctan2(py,pz)+np.radians(90)

    c, s = np.cos(theta), np.sin(theta)
    R = np.matrix('{} {}; {} {}'.format(c, -s, s, c))
    for ix, iy, iw in zip(x, y, w):
        # rotation in eta-phi plane:
        #rot = R*np.matrix([[ix],[iy]])
        #rix, riy = rot[0,0], rot[1,0]
        # rotation by lorentz transformation
        px = iw * np.cos(iy)
        py = iw * np.sin(iy)
        pz = iw * np.sinh(ix)
        rot = R*np.matrix([[py],[pz]])
        rix, riy = np.arcsinh(rot[1,0]/iw), np.arcsin(rot[0,0]/iw)
        rot_x.append(rix)
        rot_y.append(riy)

    # now reflect if leftSum > rightSum
    leftSum = 0
    rightSum = 0
    for ix, iy, iw in zip(x, y, w):
        if ix > 0:
            rightSum += iw
        elif ix < 0:
            leftSum += iw
    if leftSum > rightSum:
        ref_x = [-1.*rix for rix in rot_x]
        ref_y = rot_y
    else:
        ref_x = rot_x
        ref_y = rot_y

    return np.array(ref_x), np.array(ref_y)



In [11]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm 
    return x / (K.sqrt(K.mean(K.square(x)))+1e-5)

In [ ]:
list_x = []
list_y = []
list_w = []
njets = 0
# 4D tensor
# 1st dim is jet index
# 2nd dim is pt value (or rgb, etc.)
# 3rd dim is eta bin
# 4th dim is phi bin
if K.image_dim_ordering()=='tf':
    jet_images = np.zeros((len(df_dict_jets), nx, ny, 1))
else:
    jet_images = np.zeros((len(df_dict_jets), 1, nx, ny))

for i in range(0, len(df_dict_jets)):
    njets+=1
    #get the ith jet 
    df_dict_cand_i = df_dict_cands[(df_dict_cands['ak5pfcand_ijet'] == df_dict_jets['ak5pfcand_ijet'].iloc[i]) &
                                   (df_dict_cands['event'] == df_dict_jets['event'].iloc[i]) &
                                   (df_dict_cands['jet_pt'] == df_dict_jets['jet_pt'].iloc[i])]
    x = df_dict_cand_i['ak5pfcand_eta']-df_dict_cand_i['ak5pfcand_eta'].iloc[0]
    y = df_dict_cand_i['ak5pfcand_phi']-df_dict_cand_i['ak5pfcand_phi'].iloc[0]
    weights = df_dict_cand_i['ak5pfcand_pt'] # pt of candidate is the weight
    #x,y = rotate_and_reflect(x,y,weights)
    list_x.append(x)
    list_y.append(y)
    list_w.append(weights)
    hist, xedges, yedges = np.histogram2d(x, y,weights=weights, bins=(xbins,ybins))
    for ix in range(0,nx):
        for iy in range(0,ny):
            if K.image_dim_ordering()=='tf':
                jet_images[i,ix,iy,0] = hist[ix,iy]
            else:
                jet_images[i,0,ix,iy] = hist[ix,iy]
all_x = np.concatenate(list_x)
all_y = np.concatenate(list_y)
all_w = np.concatenate(list_w)
all_w = 1.*all_w/njets # to get average
plt.figure('W')
plt.hist2d(all_x, all_y, weights=all_w, bins=(xbins,ybins), norm=mpl.colors.LogNorm())
plt.colorbar()
plt.xlabel('eta')
plt.ylabel('phi')
plt.show()